All the dependencies

In [1]:
import time            
import re            
import os
import sys  
import codecs  
import shutil
import urllib 
from selenium import webdriver        
from selenium.webdriver.common.keys import Keys        
import selenium.webdriver.support.ui as ui        
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
from tqdm import tqdm
import pickle

In [2]:
# defining driver for the operation, we use the firefox browser with geckodriver installed
driver = webdriver.Firefox()
# not sure why we need to wait explicitly for 10 seconds here though
wait = ui.WebDriverWait(driver,10)

In [3]:
def LoginWeibo(username, password):
    try:
        #输入用户名/密码登录
        print(u'准备登陆Weibo.cn网站...')
        driver.get("http://login.sina.com.cn/")
        elem_user = driver.find_element_by_name("username")
        elem_user.send_keys(username) #用户名
        elem_pwd = driver.find_element_by_name("password")
        elem_pwd.send_keys(password)  #密码
        #elem_rem = driver.find_element_by_name("safe_login")
        #elem_rem.click()             #安全登录
 
        #重点: 暂停时间输入验证码(http://login.weibo.cn/login/ 手机端需要)
        time.sleep(20)
        
        #elem_sub = driver.find_element_by_xpath("//input[@class='smb_btn']")
        #elem_sub.click()              #点击登陆 因无name属性
        #如果登陆按钮采用动态加载 则采用输入回车键登陆微博
        elem_pwd.send_keys(Keys.RETURN)
        time.sleep(2)
        
        #获取Coockie 推荐资料：http://www.cnblogs.com/fnng/p/3269450.html
        print(driver.current_url)
        print(driver.get_cookies())  #获得cookie信息 dict存储
        print(u'输出Cookie键值对信息:')
        for cookie in driver.get_cookies(): 
            for key in cookie:
                print(key, cookie[key])
                    
        #driver.get_cookies()类型list 仅包含一个元素cookie类型dict
        print(u'登陆成功...')
        
        
    except Exception as e:      
        print("Error: ",e)
    finally:    
        print(u'End LoginWeibo!\n\n')

In [4]:
# authentication details
username = 'ruoyzhang@outlook.com'
password = 'Cyfloel1.1.'

In [5]:
LoginWeibo(username, password)

准备登陆Weibo.cn网站...
https://login.sina.com.cn/signup/signin.php
[{'name': 'ULOGIN_IMG', 'value': 'gz-090b72c97b406d09b54e37a0957c7b7482eb', 'path': '/', 'domain': '.sina.com.cn', 'secure': False, 'httpOnly': False}]
输出Cookie键值对信息:
name ULOGIN_IMG
value gz-090b72c97b406d09b54e37a0957c7b7482eb
path /
domain .sina.com.cn
secure False
httpOnly False
登陆成功...
End LoginWeibo!





Now from here on let's try out how we can execute advanced search by using 's.weibo.com'

In [6]:
# navigate to the search interface
driver.get("http://s.weibo.com/")

# set search key value
key = '人工智能'

# input the search key and hit enter in order to advance to the advanced search interface
item_inp = driver.find_element_by_xpath("//input[@type='text']")
item_inp.send_keys(key)
item_inp.send_keys(Keys.RETURN)

#wait
time.sleep(8)

# now we try to instruct selenium to click on the 'advacned search' bottom
driver.find_element_by_xpath("//a[@node-type='advsearch']").click()

NoSuchElementException: Message: Unable to locate element: //a[@node-type='advsearch']


In [8]:
driver.get('https://s.weibo.com')

In [9]:
# set search key value
key = '人工智能'

# input the search key and hit enter in order to advance to the advanced search interface
item_inp = driver.find_element_by_xpath("//input[@type='text']")
item_inp.send_keys(key)
item_inp.send_keys(Keys.RETURN)

now we try to enter the specific search values, we take the below example


key: 人工智能

time: 01/04/2019 - 07/04/2019

In [85]:
begin_date = '2018-12-30'
begin_year = str(int(begin_date.split('-')[0]))
begin_month = str(int(begin_date.split('-')[1])-1)
begin_day = str(int(begin_date.split('-')[2]))


end_date = '2019-1-1'
end_year = str(int(end_date.split('-')[0]))
end_month = str(int(end_date.split('-')[1])-1)
end_day = str(int(end_date.split('-')[2]))

In [86]:
# below for instructing selenium to click on the 'time' input field
driver.find_element_by_xpath("//dd/input[@value='请选择日期'][1]").click()

In [87]:
# below for instructing selenium to select the right **YEAR** value
driver.find_element_by_xpath("//div[@class='selector']/select[@class='year']").click()
driver.find_element_by_xpath("//div[@class='selector']/select[@class='year']/option[@value={}]".format(begin_year)).click()

In [88]:
# below for instructing selenium to select the right **MONTH** value
driver.find_element_by_xpath("//div[@class='selector']/select[@class='month']").click()
driver.find_element_by_xpath("//div[@class='selector']/select[@class='month']/option[@value={}]".format(begin_month)).click()

In [89]:
# below for instructing selenium to select the right **DAY** value
driver.find_element_by_xpath("//ul[@class='days']/li/a[@title='{}']".format(begin_date)).click()

Begin date input was successful, now let's replicate the strategy for end date

In [90]:
driver.find_element_by_xpath("//dd/input[@value='请选择日期'][2]").click()

In [91]:
driver.find_element_by_xpath("//div[@class='selector']/select[@class='year']").click()
driver.find_element_by_xpath("//div[@class='selector']/select[@class='year']/option[@value={}]".format(end_year)).click()

In [92]:
driver.find_element_by_xpath("//div[@class='selector']/select[@class='month']").click()
driver.find_element_by_xpath("//div[@class='selector']/select[@class='month']/option[@value={}]".format(end_month)).click()

In [93]:
driver.find_element_by_xpath("//ul[@class='days']/li/a[@title='{}']".format(end_date)).click()

In [94]:
# click on the 'search' button
driver.find_element_by_xpath("//div[@class='m-adv-search']/div[@class='btn-box']/a[@class='s-btn-a']").click()

## Now we move onto the actual scrapping part

In [95]:
tweets = driver.find_elements_by_xpath("/html/body/div/div/div/div/div/div/div/div/div/p[@class='txt' and @node-type='feed_list_content']")

In [96]:
for i, tweet in enumerate(tweets):
    print('******',str(i),'******')
    print(tweet.text)
    print(' ')
    print('----------------------------')
    print(' ')

****** 0 ******
冷的我想给鼠标加热，冷的我在床上想剪视频都不太可能太冷了。
但是又珍惜雪的出现比较太热就是宣判我死刑。虽然我喜欢沙滩夏日，可是这里没有沙滩就不需要夏日了吧。
北极熊是自然的超级宠儿明明拥有最完美的冰雪世界，鲸也是在最深的海底与枪乌贼的世界大战。
都是属于完美主义，可是都被全球变暖破 展开全文c
 
----------------------------
 
****** 1 ******
国中创投首席合伙人、CEO 施安平
2019年，国中创投将会继续坚持“价值投资”的理念。
明年，随着科创板的推出，中国创投行业将会迎来一个新的历史机遇。届时，可能会有大量资金涌入创投行业，这有利于解决“募资难”。我们要抓住这个机会，为被投企业走向科创板做好服务。
2019年，我看好的方向：人工 展开全文c
 
----------------------------
 
****** 2 ******
君联资本董事总经理、总裁 陈浩
2019年，在投资节奏上，君联资本将从谨慎、稳健走向更积极。对于投资阶段，我们的理想状态是早期和中后期平衡，各占50%。在退出上，借助不同资本市场的周期性，退出工作常态化。
接下来，君联资本将重点关注三个方向：
第一、新材料新工艺带动下的先进制造业；
第二、大 展开全文c
 
----------------------------
 
****** 3 ******
#天天兔兔65#　　谭天启援引实例，教育他：“马上年末了，投资总监压力最大，他们都在等排名、等净值。我刚投了人工智能医疗板块，就是你最不看好的那几家公司。公司经营得好，股票收益不一定好，公司一滩烂泥，股票收益不一定差。你总把这两个概念弄混，自视甚高，听不进劝。你坐这位置，还不如 展开全文c
 
----------------------------
 
****** 4 ******
AI+IoT”是未来的风口，也是小米核心战略之一。今天，人类进入了人工智能新时代，无论走到哪里，哪个会议都在讨论人工智能，人工智能时代远远超过以前我们提的大数据时代和移动互联网时代。
在这个时代面前，不仅仅是巨头，在十多个领域里面1000多家的创业公司风起云涌，越来越多的实用性产品开始产生 展开全文c
 
----------------------------
 
**

In [97]:
len(tweets)

20

So we have 20 tweets per page, and we have 50 pages of displayed results, this means a total of: **1000** tweets per search.

HOWEVER, we notice a bunch of duplicated contents, no real added value, we consider doing the following:

* Deduplication per page??  --> based on observation, this will not be very effective since we observe duplicated content across the pages
* Deduplication per week --> this will definitely be more effective, however it requires me to run similarity calculations again:
    1. strategy 1: cosine similarity with a computation complexity of n^2/2
    2. stragey 2: Local sensitive hashing and min hash, need to brush up on the implementation I don't have the old code

#### let's test out stategy 1 and see how much time it would take per week

First of all, let's scrape off the content and store for a small sample (1000 tweets)

extracting dates and other relevant information / fields

In [154]:
# the below extracts date
item_tweet_dates = driver.find_elements_by_xpath("/html/body/div/div/div/div/div/div/div/div/div/p[@class='from']")

In [155]:
# trying to convert string to datetime format
# first, replace the chinese characters
item_tweet_dates = [date.text.split(' ')[0].replace('年', ' ').replace('月', ' ').replace('日', '') for date in item_tweet_dates]

In [156]:
# then add year if necessary
item_tweet_dates = ['2019 ' + date if len(date) == 5 else date for date in item_tweet_dates]

In [157]:
item_tweet_dates

['2019 01 01',
 '2019 01 01',
 '2019 01 01',
 '2019 01 01',
 '2019 01 01',
 '2019 01 01',
 '2019 01 01',
 '2019 01 01',
 '2019 01 01',
 '2019 01 01',
 '2018 12 31',
 '2018 12 31',
 '2018 12 31',
 '2018 12 31',
 '2018 12 31',
 '2018 12 31',
 '2018 12 31',
 '2018 12 31',
 '2018 12 31',
 '2018 12 31']

In [158]:
# then we convert it to datetime format
item_tweet_dates = [datetime.strptime(date, '%Y %m %d') for date in item_tweet_dates]

In [143]:
len(item_tweet_dates)

20

turning the page

In [176]:
driver.find_element_by_xpath("/html/body/div/div/div/div/div/div/a[@class='next']").click()

YUS

Determining the max number of page (just in case)

In [171]:
dropdown_page = driver.find_elements_by_xpath("/html/body/div/div/div/div/div/div/span/ul[@node-type='feed_list_page_morelist']/li")

In [173]:
len(dropdown_page)

50

In [175]:
[4,5,8] + [1,2,5,6,6]

[4, 5, 8, 1, 2, 5, 6, 6]

In [178]:
path0 = '.../Documents/'
path1 = 'PythonWorkingDirectory'
os.path.join(path0, path1)

'.../Documents/PythonWorkingDirectory'

Prototyping complete:

***weibo_scrapper.py***

we'll open another notebook for testing

### We ran into an unknown problem with the login method

We now proceed to create a new login method

In [53]:
# gavigate to the appropriate page
driver.get("https://www.weibo.com/login.php")

time.sleep(5)

# click on the login button
driver.find_element_by_xpath("/html/body/div/div/div/div/div/div/div/div/ul/li/a[@node-type='loginBtn']").send_keys(Keys.RETURN)

time.sleep(3)

In [54]:
# inputting username
elem_user = driver.find_element_by_xpath("/html/body/div/div/div/div/div/input[@node-type='username']")
elem_user.clear()
elem_user.send_keys(username)

In [55]:
# inputting password
elem_pwd = driver.find_element_by_xpath("/html/body/div[4]/div[2]/div[3]/div[3]/div[2]/input")
elem_pwd.clear()
elem_pwd.send_keys(password)

In [57]:
# click on the login button
driver.find_element_by_xpath("/html/body/div/div/div/div/div/a[@node-type='submitBtn']").click()